In [2]:
import sys
sys.path.append("../mypkg")

In [3]:
from constants import RES_ROOT, FIG_ROOT, DATA_ROOT
from utils.misc import load_pkl, save_pkl, merge_intervals
from utils.colors import qual_cmap
from utils.stats import weighted_quantile
from data_gen_utils.data_gen_my3 import get_simu_data
from utils.utils import MyDataSet, get_idx_sets
from demo_settings import simu_settings
from CQR import get_CQR_CIs, boosting_pred, boosting_logi, get_CF_CIs
from ddpm.train_ddpm_now import TrainDDPM
from weighted_conformal_inference import WeightedConformalInference
from local_weighted_conformal_inference import LocalWeightedConformalInference, get_opth
from naive_sample import NaiveSample

In [4]:
%load_ext autoreload
%autoreload 2
# 0,1, 2, 3, be careful about the space

In [5]:
import torch
import scipy.stats as ss
import numpy as np
from easydict import EasyDict as edict
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict as ddict
from tqdm import tqdm, trange
import random
from joblib import Parallel, delayed
import pandas as pd
from pprint import pprint
from copy import deepcopy
plt.style.use(FIG_ROOT/"base.mplstyle")

# Params

In [6]:
setting = "setting1"

'setting1'

In [7]:
params = edict()

params.simu_setting = edict()
params.simu_setting.rho = 0.9
params.simu_setting.is_homo = False
params.simu_setting.n = 3000
params.simu_setting.d = 10
params.simu_setting.ntest = 1000
params.simu_setting.cal_ratio = 0.45 # for conformal inference
params.simu_setting.val_ratio = 0.05 # for tuning network
params.simu_setting.update(simu_settings[setting])
params.simu_setting.n = 3000
params.simu_setting.d = 100 
pprint(params.simu_setting)


params.nrep = 50 # num of reptition for simulation
params.K = 40 # num of sps drawn from q(Y(1)|X)
params.save_snapshot = 500
params.df_dtype = torch.float32
params.device="cpu"
params.n_jobs = 20
params.verbose = True
params.inf_bs = 40 # the inference batch, fct x K

params.ddpm_training = edict()
# Batch size during training
params.ddpm_training.batch_size = 256 
# Number of training epochs
params.ddpm_training.n_epoch = 1000
params.ddpm_training.n_infeat = 128
# Learning rate for optimizers
params.ddpm_training.lr = 0.001
params.ddpm_training.lr_gamma = 0.5
params.ddpm_training.lr_step = 1000
params.ddpm_training.test_intv = 5
params.ddpm_training.n_T = 400 # 100
params.ddpm_training.n_upblk = 1
params.ddpm_training.n_downblk = 1
params.ddpm_training.weight_decay = 1e-2
params.ddpm_training.early_stop = False
params.ddpm_training.early_stop_dict = {"early_stop_len":50, "early_stop_eps": 5e-4}
#params.ddpm_training.betas = [0.001, 0.5]

params.wconformal = edict()
# remove too large and too small in ws/mean(ws)
params.wconformal.nwthigh = 20
params.wconformal.nwtlow = 0.05
params.wconformal.useinf = False


params.hypo_test = edict()
params.hypo_test.alpha = 0.05 # sig level

params.prefix = ""
params.save_dir = f"demo_ddpm_{setting}_test11"
if not (RES_ROOT/params.save_dir).exists():
    (RES_ROOT/params.save_dir).mkdir()

{'cal_ratio': 0.25,
 'd': 100,
 'err_type': 'norm',
 'is_homo': True,
 'n': 3000,
 'ntest': 1000,
 'rho': 0,
 'val_ratio': 0.05}


In [8]:
torch.set_default_dtype(params.df_dtype)

In [9]:
keys = ["lr", "n_infeat", "n_T", "weight_decay", "n_upblk", "n_downblk"]
def _get_name_postfix(keys, ddpm_training):
    lst = []
    for key in keys:
        if ddpm_training[key] >= 1:
            lst.append(f"{key}-{str(ddpm_training[key])}")
        else:
            lst.append(f"{key}--{str(ddpm_training[key]).split('.')[-1]}")
    return "_".join(lst)

In [10]:
from data_gen_utils.data_gen_my3 import get_simu_data
data_train = get_simu_data(n=params.simu_setting.n, 
                                   d=params.simu_setting.d, 
                                   rho=params.simu_setting.rho, 
                                   err_type=params.simu_setting.err_type);

In [22]:
rep_ix = 1
params = params
lr = 1e-1
n_infeat = 128
n_T = 400
weight_decay = 1e-2
n_blk = 1
if True:
    manualSeed = rep_ix
    random.seed(manualSeed)
    np.random.seed(manualSeed)
    torch.manual_seed(manualSeed)
    torch.use_deterministic_algorithms(True) # Needed for reproducible results
    params = edict(deepcopy(params))
    params.ddpm_training.n_upblk = n_blk
    params.ddpm_training.n_downblk = n_blk
    params.ddpm_training.weight_decay = weight_decay
    params.ddpm_training.n_T = n_T
    params.ddpm_training.lr = lr
    params.ddpm_training.n_infeat = n_infeat
    keys = ["lr", "n_infeat", "n_T", "weight_decay", "n_upblk", "n_downblk"]
    post_fix = _get_name_postfix(keys, params.ddpm_training)
    
    torch.set_default_dtype(params.df_dtype)
    torch.set_default_device(params.device)
    
    fil_name = (RES_ROOT/params.save_dir)/f"rep_{rep_ix}_{post_fix}_res.pkl"
    ofil_name = (RES_ROOT/params.save_dir)/f"rep_{rep_ix}_others_res.pkl"
        
        
    if True:
    #if not fil_name.exists():
        data_train = get_simu_data(n=params.simu_setting.n, 
                                   d=params.simu_setting.d, 
                                   rho=params.simu_setting.rho, 
                                   err_type=params.simu_setting.err_type);
        data_test = get_simu_data(n=params.simu_setting.ntest, 
                                   d=params.simu_setting.d, 
                                   rho=params.simu_setting.rho,
                                   err_type=params.simu_setting.err_type);
        
        
        cal_idxs, val_idxs, tr_idxs = get_idx_sets(all_idxs=np.where(data_train.T)[0], 
                                                       ratios = [params.simu_setting.cal_ratio, params.simu_setting.val_ratio])
                
        # get psfun
        psY = data_train.T.astype(int)
        psX = data_train.X
        fit_res = boosting_logi(psY, psX);
        def wsfun(X):
            eps=1e-10
            if isinstance(X, torch.Tensor):
                X = X.cpu().numpy()
            if X.ndim == 1:
                X = X.reshape(1, -1)
            est_ps = boosting_pred(X, fit_res)
            est_ws = 1/(est_ps+eps)
            return torch.tensor(est_ws, dtype=params.df_dtype).to(device=params.device)
            
        
        cal_X = torch.tensor(data_train.X[cal_idxs], dtype=params.df_dtype)
        cal_Y = torch.tensor(data_train.Y1[cal_idxs], dtype=params.df_dtype)
        val_X = torch.tensor(data_train.X[val_idxs], dtype=params.df_dtype)
        val_Y = torch.tensor(data_train.Y1[val_idxs], dtype=params.df_dtype)
        test_X = torch.tensor(data_test.X, dtype=params.df_dtype)
        test_Y = torch.tensor(data_test.Y1, dtype=params.df_dtype)
            
        # train q(Y(1)|X)
        data_train_ddpm = MyDataSet(Y=data_train.Y[tr_idxs], X=data_train.X[tr_idxs])
        data_val = edict()
        data_val.c = val_X
        data_val.x = val_Y
        input_params = edict(deepcopy(params.ddpm_training))
        input_params.pop("n_epoch")
        input_params.pop("early_stop")
        input_params.pop("early_stop_dict")
        myddpm = TrainDDPM(data_train_ddpm, save_dir=params.save_dir, verbose=params.verbose, prefix=f"rep{rep_ix}_{post_fix}", 
                           device=params.device,
                           **input_params);
        
        
        myddpm.train(n_epoch=params.ddpm_training.n_epoch, 
                         data_val=data_val, save_snapshot=params.save_snapshot, 
                         early_stop=params.ddpm_training.early_stop, 
                         early_stop_dict=params.ddpm_training.early_stop_dict
                         )

2024-01-23 00:40:24,751 - ddpm.train_ddpm_now - INFO - The results are saved at /data/rajlab1/user_data/jin/MyResearch/DG-CITE_paper/notebooks/../mypkg/../results/demo_ddpm_setting1_test11.
2024-01-23 00:40:24,765 - ddpm.train_ddpm_now - INFO - The params is {'lr': 0.1, 'batch_size': 256, 'device': 'cpu', 'n_T': 400, 'n_infeat': 128, 'n_upblk': 1, 'n_downblk': 1, 'betas': [0.0001, 0.02], 'lr_gamma': 0.5, 'lr_step': 1000, 'test_intv': 5, 'weight_decay': 0.01}
2024-01-23 00:40:24,766 - ddpm.train_ddpm_now - WARNING - We do not do early stop, so any args in early_stop_dict are ignored.


The num of params is 0.32m. 
Adjusting learning rate of group 0 to 1.0000e-01.


loss: 0.4591: 100%|███████████████████████████████████████████| 1000/1000 [00:46<00:00, 21.29it/s, val loss=1.56]

Adjusting learning rate of group 0 to 5.0000e-02.


In [23]:
ddpm = myddpm.ddpm
gen_params={"ddim_eta": 1}
_get_intvs_len = lambda intvs: np.array([sum([np.diff(iv) for iv in intv])[0] for intv in intvs]);
# get weather vaule in vs is in CI in intvs or not 
def _get_inset(vs, intvs):
    in_set = []
    for v, intv in zip(vs, intvs):
        in_set.append(np.sum([np.bitwise_and(v>iv[0], v<iv[1]) for iv in intv]))
    in_set = np.array(in_set)
    return in_set
wcf1 = WeightedConformalInference(cal_X, 
                                     cal_Y,
                                     ddpm, ws_fn=wsfun, verbose=2, 
                                     gen_type="ddim",
                                     seed=manualSeed,
                                     n_jobs=params.n_jobs,
                                     inf_bs=params.inf_bs,
                                     device=params.device,
                                     gen_params=gen_params,
                                     wcf_params={
                                        "K": params.K, # num of sps for each X
                                        "nwhigh" : params.wconformal.nwthigh,
                                        "nwlow" : params.wconformal.nwtlow,
                                        "useinf": params.wconformal.useinf,
                                     })
wcf2 = LocalWeightedConformalInference(cal_X, 
                                     cal_Y,
                                     ddpm, ws_fn=wsfun, verbose=2, 
                                     gen_type="ddim",
                                     seed=manualSeed,
                                     n_jobs=params.n_jobs,
                                     inf_bs=params.inf_bs,
                                     device=params.device,
                                     gen_params=gen_params,
                                     wcf_params={
                                        "K": params.K, # num of sps for each X
                                        "nwhigh" : params.wconformal.nwthigh,
                                        "nwlow" : params.wconformal.nwtlow,
                                        "useinf": params.wconformal.useinf,
                                     })


2024-01-23 00:41:26,274 - weighted_conformal_inference - INFO - wcf params is {'K': 40, 'nwhigh': 20, 'nwlow': 0.05, 'useinf': False, 'cf_type': 'PCP'}
2024-01-23 00:41:26,276 - weighted_conformal_inference - INFO - gen params is {'ddim_timesteps': 50, 'ddim_eta': 1, 'gen_type': 'ddim'}
2024-01-23 00:41:26,278 - weighted_conformal_inference - INFO - wcf params is {'K': 40, 'nwhigh': 20, 'nwlow': 0.05, 'useinf': False, 'cf_type': 'PCP'}
2024-01-23 00:41:26,279 - weighted_conformal_inference - INFO - gen params is {'ddim_timesteps': 50, 'ddim_eta': 1, 'gen_type': 'ddim'}


In [71]:
tmpX=torch.tensor(data_train.X[:1000], dtype=params.df_dtype);
opt_h = get_opth(X=tmpX, hmin=0.1, local_method="RLCP", target_eff_size=800)

6.18471177944862

In [64]:
X = test_X
Y = test_Y
intvs1 = wcf1(X, alpha=params.hypo_test.alpha);
prbs1 = np.mean(_get_inset(Y, intvs1))
mlen1 = np.median(_get_intvs_len(intvs1))
print(prbs1, mlen1)
wcf2.add_data(X)
intvs2 = wcf2(local_method="RLCP", alpha=params.hypo_test.alpha, lm_params={"h":opt_h});
prbs2 = np.mean(_get_inset(Y, intvs2))
mlen2 = np.median(_get_intvs_len(intvs2))
print(prbs2, mlen2)

100%|████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 60.81it/s]


0.957 15.522781


100%|██████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 4506.13it/s]


0.958 15.685301


In [56]:
data_test1 = get_simu_data(n=1000,
                           d=params.simu_setting.d, 
                           rho=params.simu_setting.rho,
                           err_type=params.simu_setting.err_type);
        

In [57]:
X = torch.tensor(data_test1.X, dtype=params.df_dtype);
Y = torch.tensor(data_test1.Y1, dtype=params.df_dtype);
Xnorm = torch.norm(X, p=2, dim=1)**2
kpidx = Xnorm < 100/3
X = X[kpidx]
Y = Y[kpidx];
print(X.shape)

intvs1 = wcf1(X, alpha=params.hypo_test.alpha);
prbs1 = np.mean(_get_inset(Y, intvs1))
mlen1 = np.median(_get_intvs_len(intvs1))
print(prbs1, mlen1)

wcf2.add_data(X)
intvs2 = wcf2(local_method="baseLCP", alpha=params.hypo_test.alpha, lm_params={"h":opt_h-1});
prbs2 = np.mean(_get_inset(Y, intvs2))
mlen2 = np.median(_get_intvs_len(intvs2))
print(prbs2, mlen2)

torch.Size([518, 100])


100%|██████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 6038.98it/s]


0.9459459459459459 15.599217


100%|█████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 11071.26it/s]


0.9691119691119691 17.361458


In [20]:
intvs2 = wcf2(local_method="baseLCP", alpha=params.hypo_test.alpha, lm_params={"h":5});
prbs2 = np.mean(_get_inset(Y, intvs2))
mlen2 = np.median(_get_intvs_len(intvs2))
print(prbs2, mlen2)

0.9450980392156862 49.714363


In [66]:
intvs2 = wcf2(local_method="baseLCP", alpha=params.hypo_test.alpha, lm_params={"h":0.5});

tensor([[0.2500, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2500, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.2500, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2500, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.2500]])
tensor([-58.1507, -22.5791])
tensor([-55.9703, -22.5791])
tensor([-53.3971, -22.5791])
tensor([-57.3404, -22.5791])
tensor([-52.6056, -22.5791])
tensor([-53.5187, -22.5791])
tensor([-57.3497, -22.5791])
tensor([-62.3816, -22.5791])
tensor([-56.3065, -22.5791])
tensor([-55.0109, -22.5791])
tensor([-63.3423, -22.5791])
tensor([-57.6119, -22.5791])
tensor([-54.6570, -22.5791])
tensor([-55.9860, -22.5791])
tensor([-55.4005, -22.5791])
tensor([-60.5444, -22.5791])
tensor([-63.4337, -22.5791])
tensor([-59.6194, -22.5791])
tensor([-57.2862, -22.5791])
tensor([-60.4052, -22.5791])
tensor([-53.0632, -22.

In [51]:
(2*torch.pi)**(-50)

1.2331235221003418e-40